In [1]:
import re
import pandas as pds
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split  
from sklearn import metrics


In [2]:
def cutvalue(item):
    topcut = []
    for n in item['簡介']:
        top = jieba.analyse.extract_tags(n,topK=2)
        topcut.append(top)

    typecut = []
    for n in item['類型']:
        ch =re.compile("[\u4e00-\u9fa5]")
        seg_word =  "".join(ch.findall(n))
        top = jieba.lcut(seg_word)
        typecut.append(top)

    namecut = []
    for n in item['中文名稱']:
        ch =re.compile("[\u4e00-\u9fa5]")
        name =  "".join(ch.findall(n))
        namecut.append(name)

    data = { 'type':typecut ,'name':namecut,'article':topcut}
    def1 = pds.DataFrame(data)

    def1['type'] = def1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    def1['article'] = def1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    def1['key'] = def1['name'].astype(str)+' '+def1['type'].astype(str)+' '+def1['article'].astype(str)
    return def1

In [3]:
def count(item):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(item['key'])
    tfidf = TfidfTransformer() 
    tf=tfidf.fit_transform(X)
    word = vectorizer.get_feature_names() 
    
    return tf

In [4]:
def Accuracy(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07659)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)  
    y_test=y_test.values

    print("精準度：",metrics.accuracy_score(y_test, y_pred))

In [5]:
item = pds.read_csv('movie.csv')
item =item.drop(labels=['Unnamed: 0'],axis='columns')
def1 = cutvalue(item) 
tf = count(def1)
Accuracy(tf,def1)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.644 seconds.
Prefix dict has been built successfully.


精準度： 0.724
